<center>
    <div style="font-size: 24px; font-weight: 500; margin-bottom: -10px">
        Climat et Changement Climatique
    </div>
    <h3>Master</h3>
    <h5 style="margin-top: -5px">Sciences de l’Océan, de l’Atmosphère et du Climat</h5>
    <h1>Exam 2025-2026</h1>
    <h2>Modelling the relationship between warming and CO$_2$ emissions</h2>
</center>


## 1. Introduction

In this exercise, we propose to build a Simple Climate Model to examine the relationship between CO$_2$ emissions and the global warming by combining what we have learned about the carbon cycle (Part I) and the response of temperature to a pulse of radiative forcing using CMIP5 model outputs (Part II).


> **We ask you to send this notebook completed by your _own code_ and your _answers (french or english)_ to questions by email at romain.torres4@gmail.com**
> - rename the file with your name: **TP_SOAC-Exam_2025-2026-Name.ipynb**
> - you can use internet and your previous python codes from **Part I and Part II**
> - the exam should take **one hour** and is **individual** (one file per student)

As discussed during the class, you will be evaluated on your understanding of physical concepts about climate and your python code (figures, units, array manipulations, etc.)

## 2. Questions

You first need to import and load required modules:  

In [ ]:
# header: importing useful modules and functions
import csv
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fmin

#### Question 1:

* *Use the impulse response function of CO$_2$ (Part I) to simulate the response of atmospheric CO$_2$ (in ppm) to a constant yearly emission of 10 GtCO$_2$ yr$^{-1}$ over 100 years.*
* _Plot the response of the model over time._

In [ ]:
# definition of the IRF

# WRITE YOUR CODE HERE

In [ ]:
# plot the simulation

# WRITE YOUR CODE HERE

#### Question 2:

Now we will try to model the surface temperature rise to this CO$_2$ forcings.
For that, we will use the output of the [Community Earth System Model](https://doi.org/10.1175/BAMS-D-12-00121.1) version 1.2 (CESM1).

The following python code load the model output for the _picontrol_ and _abrupt 4xCO2_ simulations.

In [ ]:
# initalise empty dict
picontrol   = dict()
abrupt4xCO2 = dict()
som_4xCO2   = dict() 

# open simulation results
ctrl_datas = np.genfromtxt('./data/CESM1/CESM1-LME-picontrol_global.csv', 
                             skip_header=1, delimiter=',')
quad_datas = np.genfromtxt('./data/CESM1/CESM1b-4xCO2_global.csv', 
                             skip_header=1, delimiter=',')
som_datas  = np.genfromtxt('./data/CESM1/CESM1b-4xCO2-SOM_global.csv', 
                             skip_header=1, delimiter=',')

# yearly averaged variables
def moving_average(a, n=12):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    
    return ret[n - 1:] / n

# -- picontrol run
with open('./data/CESM1/CESM1-LME-picontrol_global.csv', newline='') as f:
  reader = csv.reader(f)
  header = next(reader)
    
for i, v in enumerate(header[1:]):
    var_name = v.split(' ')[0]
    picontrol[ var_name ] = moving_average(ctrl_datas[:,i+1])[5::12]

# -- abrupt4xCO2 run
with open('./data/CESM1/CESM1b-4xCO2_global.csv', newline='') as f:
  reader = csv.reader(f)
  header = next(reader)
    
for i, v in enumerate(header[1:]):
    var_name = v.split(' ')[0]
    abrupt4xCO2[ var_name ] = moving_average(quad_datas[:,i+1])[5::12]

# -- Slab Ocean Model run
with open('./data/CESM1/CESM1b-4xCO2-SOM_global.csv', newline='') as f:
  reader = csv.reader(f)
  header = next(reader)
    
for i, v in enumerate(header[1:]):
    var_name = v.split(' ')[0]
    som_4xCO2[ var_name ] = moving_average(som_datas[:,i+1])[5::12]

Yearly averaged datas are stored in python dictionaries `picontrol`, `abrupt4xCO2` and `som_4xCO2`. 
You can access to the variable with the following code:
```python
# picontrol
picontrol['FSNT'] # Net solar flux at the top of the model 
picontrol['FLNT'] # Net longwave flux at the top of the model
picontrol['TS']   # Global average of the surface temperature

# abrupt 4xCO2
abrupt4xCO2['FSNT'] # Net solar flux at the top of the model 
abrupt4xCO2['FLNT'] # Net longwave flux at the top of the model
abrupt4xCO2['TS']   # Global average of the surface temperature
```

* *Using the method of [Gregory et al. (2004)](https://doi.org/10.1029/2003GL018747) you have learned (Part II), find the equilibrium climate sensitivity in the abrupt 4xCO2 experiment (ECS$_{\text{4xCO}_2}$).*

Note that since _picontrol_ simulation is already in a steady state, the pre-industrial climate state (e.g. surface temperature and radiative fluxes) can be deduced from temporal means.

In [ ]:
# WRITE YOUR CODE HERE

#### Question 3:

The method employed extrapolate the results from fully coupled atmosphere-ocean climate models. This extrapolation is necessary since these models take millennia to equilibriate. 

Another approach is the use of a thermodynamic "slab" ocean model (SOM) rather than a full dynamical ocean in abrupt CO$_2$ increase experiments to eliminate the long time scales produced by the slow deep-ocean responses to warming. The SOM experiments equilibrate in decades rather than centuries, yielding a SOM-based estimate of ECS (ECS-SOM).

* *From the python dictionary `som_4xCO2`, propose an estimation for ECS-SOM to an abrupt 4xCO2 concentration.*

You can access to the variable with the following code:
```python
# SOM 4xCO2
som_4xCO2['FSNT'] # Net solar flux at the top of the model 
som_4xCO2['FLNT'] # Net longwave flux at the top of the model
som_4xCO2['TS']   # Global average of the surface temperature
```

In [ ]:
# WRITE YOUR CODE HERE

* *How different are ECS-SOM and ECS$_{\text{4xCO}_2}$ ? From your knowledge of climate system, what can explain the difference ?*

<font color="blue">
WRITE YOUR RESPONSE HERE !
</font>

#### Question 4:

* *Using the ECS in the abrupt 4xCO2 simulation, find the parameters $a_{T,1}$, $a_{T,2}$, $\tau_{T,1}$ and $\tau_{T,2}$ to fit the function bellow with the simulation outputs*

$$ tas(t) = \text{ECS}_{\text{4xCO}_2} \left[1 - a_{T,1}\exp(\frac{-t}{\tau_{T,1}}) - a_{T,2}\exp(\frac{-t}{\tau_{T,2}})\right]$$

In [ ]:
# fit dynamical parameters for the abrupt 4xCO2 run
# WRITE YOUR CODE HERE

* *Convert the CO$_2$ concentration into radiative Forcing F$_{\text{CO}_2}$ (see Part II) and using the derivative of the tas function (which is the IRF), compute and show the warming response over time to this constant emission.*

__Note__: the warming response is the convolution product between the IRF for tas and F$_{\text{CO}_2}$ (think about the units) 

__Help__: you can take the following value for the parameters

```python
a_T1   = 0.61
a_T2   = 0.39
tau_T1 = 3.44
tau_T2 = 233.
```

In [ ]:
# WRITE YOUR CODE HERE

The recent IPCC report gives a likely range of TCRE (transient surface temperature change per 1000 GtCO$_2$ cumulative emissions) of 0.27-0.63 ◦ C (IPCC, SPM, 2021).

* *Discuss your outcome with this range.*
* *What could explain this large range ? Do you expect another result with different models ?*

<font color="blue">
WRITE YOUR RESPONSE HERE !
</font>